In [ ]:
# HIDDEN
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [ ]:
# Some functions for plotting. You don't have to understand how any
# of the functions in this cell work, since they use things we 
# haven't learned about in Data 8.

def resize_window(lim=3.5):
    plots.xlim(-lim, lim)
    plots.ylim(-lim, lim)
    
def draw_line(slope=0, intercept=0, x=make_array(-4, 4), color='r'):
    y = x*slope + intercept
    plots.plot(x, y, color=color)
    
def draw_vertical_line(x_position, color='black'):
    x = make_array(x_position, x_position)
    y = make_array(-4, 4)
    plots.plot(x, y, color=color)
    
def make_correlated_data(r):
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    return x, y

def r_table(r):
    """
    Generate a table of 1000 x,y data points in standard units
    whose correlation is approximately equal to r
    """
    np.random.seed(8)
    x, y = make_correlated_data(r)
    return Table().with_columns('x', x, 'y', y)

In [ ]:
def demographics_errors(slope, intercept):
    # Use four convenient points from the original data
    sample = [[14.7, 33995], [19.1, 61454], [50.7, 71183], [59.5, 105918]]
    demographics.scatter('College%', 'Median Income', alpha=0.5)
    xlims = make_array(5, 75)
    # Plot a line with the slope and intercept you specified:
    plots.plot(xlims, slope * xlims + intercept, lw=4)
    # Plot red lines from each of the four points to the line
    for x, y in sample:
        plots.plot([x, x], [y, slope * x + intercept], color='r', lw=4)

In [ ]:
def show_demographics_rmse(slope, intercept):
    demographics_errors(slope, intercept)
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    prediction = slope * x + intercept
    mse = np.mean((y - prediction) ** 2)
    print("Root mean squared error:", round(mse ** 0.5, 2))

In [ ]:
def fitted_values(t, x, y):
    """Return an array of the regressions estimates at all the x values"""
    a = slope(t, x, y)
    b = intercept(t, x, y)
    return a*t.column(x) + b

## Slope & Intercept

In [ ]:
def standard_units(x):
    #"""Converts an array x to standard units"""
    return (x - np.mean(x)) / np.std(x)

def correlation(t, x, y):
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.mean(x_su * y_su)


In [ ]:
def slope(t, x, y):
    #""" Computes the slope of the regression line, like correlation above """
    r = correlation(t, x, y)
    y_sd = np.std(t.column(y))
    x_sd = np.std(t.column(x))
    return r * y_sd / x_sd

    
def intercept(t, x, y):
    """ Computes the intercept of the regression line, like slope above """
    x_mean = np.mean(t.column(x))
    y_mean = np.mean(t.column(y))
    return y_mean - slope(t, x, y)*x_mean

In [ ]:
example = r_table(0.5)
slope(example, 'x', 'y')

## Heights data

In [ ]:
families = Table.read_table('heights.csv')

heights = Table().with_columns(
    'MidParent', families.column('midparentHeight'),
    'Child', families.column('childHeight'))
heights

In [ ]:
def nn_prediction_heights(h):
    """
    Return a prediction of the height of a child 
    whose parents have a midparent height of h.
    
    The prediction is the average height of the children 
    whose midparent height is in the range h plus or minus 0.5 inches.
    """
    neighbors = heights.where(
        'MidParent', are.between(h - 0.5, h + 0.5))
    return np.mean(neighbors.column('Child'))

In [ ]:
heights_with_predictions = heights.with_column(
    'Average neighbor prediction', 
    heights.apply(nn_prediction_heights, 'MidParent'))

In [ ]:
heights_slope = slope(heights, 'MidParent', 'Child')
heights_intercept = intercept(heights, 'MidParent', 'Child')
heights_slope, heights_intercept

Write these values as a linear regression model. 

y=

Interpret the slope and intercept in the context of the data. 


In [ ]:
heights_with_predictions = heights_with_predictions.with_column(
    'Regression Prediction', 
    heights_slope*heights.column('MidParent') + heights_intercept
)
heights_with_predictions

In [ ]:
heights_with_predictions.scatter('MidParent')

### Discussion Question

A course has a midterm (average 70; standard deviation 10)</br>
and a really hard final (average 50; standard deviation 12).</br>

If the scatter diagram comparing midterm & final scores for 
students has an oval shape with correlation 0.75, then... </br>

What do you expect the average final score would be for </br>
students who scored 90 on the midterm?</br>

How about 60 on the midterm?


In [ ]:
mean_x = 70 
sd_x = 10
mean_y = 50
sd_y = 12

r = 0.75

#Question 1
x = 90

num_sds_x = (x - mean_x)/sd_x
num_sds_y_estimate = r * num_sds_x

y_pred = num_sds_y_estimate * sd_y + mean_y

y_pred

In [ ]:
#Using linear regression, find  the slope and the intercept to predict y

slope2 = r * sd_y/sd_x

#y = mx + b

slope2 * 90 + (mean_y - slope2 * mean_x)


In [ ]:
#Predict the answer for 60 on the midterm. 
x = 60

num_sds_x = (x - mean_x)/sd_x
num_sds_y_estimate = r * num_sds_x

y_pred = num_sds_y_estimate * sd_y + mean_y

y_pred

### Error in Estimation

Prediction has an inherrent error - the difference from the actual observed value at any given x.

The Regressions Lines is the one line that minimizes the mean squared error of estimation. 

In [ ]:
demographics = Table.read_table('district_demographics2016.csv')
demographics.show(5)

In [ ]:
demographics = demographics.drop(
    'State', 'District', 'Percent voting for Clinton')
demographics.show(5)

In [ ]:
#Compare College% and Median Income in a scatter. 



In [ ]:
#Find the correlation coefficient, r, of the data in the scatter. 
correlation(demographics, 'College%', 'Median Income')

In [ ]:
regression_slope = slope(demographics, 'College%', 'Median Income')
regression_intercept = intercept(demographics, 'College%', 'Median Income')
regression_slope, regression_intercept

Write these as a linear regression model.

y= 

Interpret the slope and intercept in the context of the data. 


In [ ]:
predicted = fitted_values(demographics, 'College%', 'Median Income')

In [ ]:
demographics = demographics.with_column(
    'Linear Prediction', predicted)
demographics.scatter('College%')

In [ ]:
#Measuring the errors. 

actual = demographics.column('Median Income')
errors = actual - predicted

#### Measuring the Errors

Since some of the predictions are above the observed values and some are below, we will take the mean of the quared errors to avoid cancellation. 
Then take the square root of the mean of the squared errors to get the root mean squared error. 

This is the value we want to minimize. 

In [ ]:
#Add the errors column to the demographics table. Do not reassign.



In [ ]:
#Find the mean of the erros
np.mean(errors)

In [ ]:
#Take the square root of the mean of the errors squared. 
np.sqrt(np.mean(errors ** 2))

In [ ]:
#Visualizing the errors. Using the linear regression line. 

demographics_errors(regression_slope, regression_intercept)

In [ ]:
# Takes any slope, any intercept. Using a different slope and y-intercept.

demographics_errors(1500, 20000)

In [ ]:
# Another slope and y intercept. 

demographics_errors(-1000, 75000)

### Root Mean Square Error ###

In [ ]:
#Measure the error from the given line. Function defined above. 

show_demographics_rmse(-1000, 75000)

In [ ]:
show_demographics_rmse(1500, 20000)

In [ ]:
#Using the slope and intercept of the regression line gives the least error. 
show_demographics_rmse(regression_slope, regression_intercept)

### Numerical Optimization ###

Optimization, typically learned in PreCalculus and Calculus, allows us to find the maximum or minimum value of a function. </br>
We want to minimize the error or the root mean squared error (RMSE).

In [ ]:
x = np.arange(1, 3, 0.1)
y = (x-2)**2 + 3
Table().with_columns('x', x, 'y', y).plot('x')

In [ ]:
def f(x):
    return ((x-2)**2) + 3

In [ ]:
minimize(f)

In [ ]:
x = np.arange(-1.5, 1.5, 0.05)
y2 = 2 * np.sin(x*np.pi) + x ** 3 + x ** 4 
Table().with_columns('x', x, 'y', y2).plot('x')

In [ ]:
def complicated_function(x):
    return 2 * np.sin(x*np.pi) + x ** 3 + x ** 4 

In [ ]:
minimize(complicated_function)

### Minimizing RMSE ###

In [ ]:
# Creating a function that will take in any slope and intercept and five the minimum error based on that estimate. 
def demographics_rmse(any_slope, any_intercept):
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    estimate = any_slope*x + any_intercept
    return np.sqrt(np.mean((y - estimate) ** 2))

In [ ]:
demographics_rmse(1500, 20000)

In [ ]:
demographics_rmse(-1000, 75000)

In [ ]:
# Minimizing all possible values

minimize(demographics_rmse)

In [ ]:
#Compare to the regression line slope and intercept.
make_array(regression_slope, regression_intercept)

Write this as a linear regression model. 

y= ...

Interpret the slope and intercept in the context of the data. 


### Nonlinear Regression ###

We can find a best fit line for non linear data, it may not be in the form $y=mx+b$. </br>
Some typical forms could be:</br>
quadratic: $y=ax^2 + bx+ c$</br>
Cubic: $y=ax^3 + bx^2+ cx$+d$

In [ ]:
shotput = Table.read_table('shotput.csv')
shotput

In [ ]:
shotput.scatter('Weight Lifted')

In [ ]:
def shotput_linear_rmse(any_slope, any_intercept):
    x = shotput.column('Weight Lifted')
    y = shotput.column('Shot Put Distance')
    estimate = any_slope*x + any_intercept
    return np.sqrt(np.mean((y - estimate) ** 2))

In [ ]:
best_line = minimize(shotput_linear_rmse)
best_line

In [ ]:
weights = shotput.column(0)

In [ ]:
linear_fit = best_line.item(0)*weights + best_line.item(1)

shotput.with_column(
    'Best Line', linear_fit
).scatter(0)

**Quadratic Function**

$$
f(x) ~=~ ax^2 + bx + c
$$
for constants $a$, $b$, and $c$.



In [ ]:
def shotput_quadratic_rmse(a, b, c):
    x = shotput.column('Weight Lifted')
    y = shotput.column('Shot Put Distance')
    estimate = a*(x**2) + b*x + c
    return np.sqrt(np.mean((y - estimate) ** 2))

In [ ]:
best_quad = minimize(shotput_quadratic_rmse)
best_quad

In [ ]:
# x = weight lifted = 100 kg
# Then predicted shot put distance:

(-0.00104)*(100**2) + 0.2827*100 - 1.5318

In [ ]:
quad_fit = best_quad.item(0)*(weights**2) + best_quad.item(1)*weights + best_quad.item(2)

In [ ]:
shotput.with_column('Best Quadratic Curve', quad_fit).scatter(0)